In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Conv2D,MaxPooling2D
import pickle
from tensorflow.keras.callbacks import TensorBoard
import time

#Name="MRI-BrainTumor-{}".format(int(time.time()))

#tensorboard = TensorBoard(log_dir='logs\\{}'.format(Name))

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

##normalizing features and labels
X= np.array(X/255.0)##has one channel. 180by180 961
y = np.array(y)

dense_layers=[0]
layer_sizes=[128]
conv_layers=[1]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            Name = "{}-conv-{}-nodes-{}-dense{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            print(Name)
            tensorboard = TensorBoard(log_dir='logs3\\{}'.format(Name))
            model = Sequential()
            ## conv layer
            model.add(Conv2D(layer_size,(3,3),input_shape = X.shape[1:]))##[1:] here is passing the channel and the img size
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(64))
                model.add(Activation("sigmoid"))
                #In simple terms, a dropout means that those neurons, which are selected at random, are ignored by the units 
                model.add(Dropout(0.2))
            
            

            #model.add(Dense(64))
            #model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss="binary_crossentropy",
                        optimizer="adam", ##gradient desc
                         metrics=['accuracy'])
            model.fit(X,y,batch_size=50,epochs=20,validation_split=0.2,callbacks=[tensorboard])
model.save('Covid.model')


1-conv-128-nodes-0-dense1618107327
Train on 768 samples, validate on 193 samples
Epoch 1/20
768/768 [==============================] - 34s 45ms/sample - loss: 1.8902 - accuracy: 0.9128 - val_loss: 0.2332 - val_accuracy: 0.9378
Epoch 2/20
768/768 [==============================] - 29s 38ms/sample - loss: 0.3050 - accuracy: 0.9245 - val_loss: 0.2488 - val_accuracy: 0.9378
Epoch 3/20
768/768 [==============================] - 29s 37ms/sample - loss: 0.2110 - accuracy: 0.9375 - val_loss: 0.1883 - val_accuracy: 0.9378
Epoch 4/20
768/768 [==============================] - 29s 37ms/sample - loss: 0.1604 - accuracy: 0.9375 - val_loss: 0.1472 - val_accuracy: 0.9378
Epoch 5/20
768/768 [==============================] - 29s 38ms/sample - loss: 0.1259 - accuracy: 0.9479 - val_loss: 0.1493 - val_accuracy: 0.9378
Epoch 6/20
768/768 [==============================] - 29s 38ms/sample - loss: 0.1051 - accuracy: 0.9635 - val_loss: 0.1164 - val_accuracy: 0.9793
Epoch 7/20
768/768 [=======================

In [2]:
################ K Nearest Neighbor #####################
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X= np.array(X/255.0)
y = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=1)
#print(X_train.shape)
#print(X_test.shape)
X_train = X_train.reshape(768,1*180*180)
X_test = X_test.reshape(193,1*180*180)
#y_train = X_train.reshape(202,1*100*100)
#y_test = X_test.reshape(51,1*100*100)
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train,y_train)

y_pred=classifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.75      0.75      0.75        12
           1       0.98      0.98      0.98       181

    accuracy                           0.97       193
   macro avg       0.87      0.87      0.87       193
weighted avg       0.97      0.97      0.97       193

[[  9   3]
 [  3 178]]


In [3]:
################ LOGISTIC REGRESSION #####################
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X= np.array(X/255.0)
y = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=1)
#print(X_train.shape)
#print(y_test.shape)
X_train = X_train.reshape(768,1*180*180)
X_test = X_test.reshape(193,1*180*180)
#y_train = X_train.reshape(202,1*100*100)
#y_test = X_test.reshape(51,1*100*100)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

y_pred=classifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.97      0.99      0.98       181

    accuracy                           0.96       193
   macro avg       0.88      0.79      0.82       193
weighted avg       0.96      0.96      0.96       193

[[  7   5]
 [  2 179]]


C:\Users\RAMBO\anaconda3\envs\python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
